<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>

# Google GenAI SDK - Building a Parallelization Agent

## Install Dependencies

In [ ]:
!pip install -q google-genai

In [ ]:
from google import genai

## Authenticate with Google Vertex AI

In [ ]:
!gcloud auth login

In [ ]:
# Create a client using the Vertex AI API, you could also use the Google GenAI API instead here
client = genai.Client(vertexai=True, project="sandbox-455622", location="us-central1")

# Parallelization

In [ ]:
import concurrent.futures
import textwrap

GEMINI_MODEL_NAME = "gemini-2.0-flash-001"

# --- 1. Define Research Prompts ---
# We define the prompts directly, without ADK agent wrappers.

research_prompts = {
    "ai_research_result": (
        "Artificial Intelligence",
        """You are an AI Research Assistant.
Research the latest advancements in 'Artificial Intelligence'.
Summarize your key findings concisely (1-2 sentences).
Focus on information readily available up to your knowledge cutoff.
Output *only* the summary.""",
    ),
    "quantum_research_result": (
        "Quantum Computing",
        """You are an AI Research Assistant specializing in physics and computing.
Research the latest breakthroughs in 'Quantum Computing'.
Summarize your key findings concisely (1-2 sentences).
Focus on information readily available up to your knowledge cutoff.
Output *only* the summary.""",
    ),
    "biotech_research_result": (
        "Biotechnology",
        """You are an AI Research Assistant specializing in life sciences.
Research the latest innovations in 'Biotechnology'.
Summarize your key findings concisely (1-2 sentences).
Focus on information readily available up to your knowledge cutoff.
Output *only* the summary.""",
    ),
}

# --- 2. Execute Research Tasks in Parallel ---


# Function to call the Gemini API for a single research task
def run_research_task(topic, prompt):
    """Calls the generative model and returns the text result."""
    print(f"Starting research for: {topic}...")
    try:
        response = client.models.generate_content(model=GEMINI_MODEL_NAME, contents=prompt)
        print(f"Finished research for: {topic}.")
        return response.text.strip()
    except Exception as e:
        print(f"Error during research for {topic}: {e}")
        return f"Error retrieving information for {topic}."


# Use ThreadPoolExecutor for parallel execution
research_results = {}
with concurrent.futures.ThreadPoolExecutor() as executor:
    # Submit tasks
    future_to_key = {
        executor.submit(run_research_task, topic, prompt): key
        for key, (topic, prompt) in research_prompts.items()
    }
    # Collect results as they complete
    for future in concurrent.futures.as_completed(future_to_key):
        key = future_to_key[future]
        try:
            result = future.result()
            research_results[key] = result
        except Exception as exc:
            print(f"{key} generated an exception: {exc}")
            research_results[key] = f"Error in {key} task."

print("\n--- Parallel Research Results ---")
for key, result in research_results.items():
    print(f"{key}: {result}")

# --- 3. Define and Execute the Synthesis Task ---

# Prepare the synthesis prompt using the collected results
synthesis_instruction = f"""You are an AI Assistant responsible for combining research findings into a structured report.

Your primary task is to synthesize the following research summaries, clearly attributing findings to their source areas (AI, Quantum Computing, Biotechnology). Structure your response using headings for each topic. Ensure the report is coherent and integrates the key points smoothly.

**Crucially: Your entire response MUST be grounded *exclusively* on the information provided in the 'Input Summaries' below. Do NOT add any external knowledge, facts, or details not present in these specific summaries.**

**Input Summaries:**

*   **AI Advancements:** {research_results.get('ai_research_result', 'N/A')}
*   **Quantum Computing:** {research_results.get('quantum_research_result', 'N/A')}
*   **Biotechnology:** {research_results.get('biotech_research_result', 'N/A')}

Produce the final synthesized report.
"""

print("\n--- Starting Synthesis ---")
try:
    synthesis_response = client.models.generate_content(
        model=GEMINI_MODEL_NAME, contents=synthesis_instruction
    )
    final_report = synthesis_response.text.strip()
    print("--- Synthesis Complete ---")
except Exception as e:
    print(f"Error during synthesis: {e}")
    final_report = "Error generating the final report."


# --- 4. Display the Final Report ---
print("\n=== Final Synthesized Report ===\n")
# Use textwrap for potentially long reports in notebooks
print(textwrap.fill(final_report, width=80))